In [3]:
import cv2  # استيراد مكتبة OpenCV للعمل مع الصور والفيديو
import numpy as np  # استيراد مكتبة Numpy للعمل مع المصفوفات
import threading  # استيراد مكتبة threading لإنشاء وتشغيل الخيوط

# تحميل YOLO
# هنا نقوم بتحميل الشبكة العصبية YOLO والملفات الخاصة بها (الوزن والتكوين)
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# الحصول على أسماء الطبقات من الشبكة العصبية
layer_names = net.getLayerNames()
# تحديد أسماء الطبقات النهائية التي سنستخدمها للكشف عن الكائنات
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# تحميل أسماء الأصناف (الأشياء التي يمكن لـ YOLO التعرف عليها)
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# تهيئة الكاميرا
cap = cv2.VideoCapture(0)  # فتح الكاميرا

# دالة لمعالجة الإطارات
def process_frame(frame):
    height, width, channels = frame.shape  # الحصول على أبعاد الإطار (الصورة)
    # تحويل الصورة إلى تنسيق blob المناسب لإدخالها في الشبكة العصبية
    blob = cv2.dnn.blobFromImage(frame, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)  # إدخال blob إلى الشبكة العصبية
    outs = net.forward(output_layers)  # الحصول على نتائج الكشف من الشبكة العصبية
    
    class_ids = []  # قائمة لتخزين معرفات الأصناف المكتشفة
    confidences = []  # قائمة لتخزين درجات الثقة للكشف
    boxes = []  # قائمة لتخزين مربعات الإحاطة للكائنات المكتشفة

    # تحليل نتائج الكشف
    for out in outs:
        for detection in out:
            scores = detection[5:]  # الحصول على الدرجات (الاحتمالات) لكل صنف
            class_id = np.argmax(scores)  # الحصول على معرف الصنف الأعلى احتمالاً
            confidence = scores[class_id]  # درجة الثقة للصنف
            if confidence > 0.5:  # إذا كانت درجة الثقة أكبر من 50%
                # حساب إحداثيات مربع الإحاطة للكائن المكتشف
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])  # إضافة مربع الإحاطة إلى القائمة
                confidences.append(float(confidence))  # إضافة درجة الثقة إلى القائمة
                class_ids.append(class_id)  # إضافة معرف الصنف إلى القائمة

    # تطبيق NMS لتجنب التكرار
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    return boxes, confidences, class_ids, indexes  # إرجاع القيم المكتشفة

# بدء الحلقة الرئيسية لقراءة ومعالجة الإطارات من الكاميرا
while True:
    ret, frame = cap.read()  # قراءة إطار من الكاميرا
    if not ret:
        break  # إذا لم نتمكن من قراءة الإطار، نخرج من الحلقة
    
    # إنشاء خيط لمعالجة الإطار
    thread = threading.Thread(target=process_frame, args=(frame,))
    thread.start()  # بدء تشغيل الخيط
    thread.join()  # انتظار انتهاء الخيط من العمل
    
    # استدعاء دالة المعالجة للحصول على نتائج الكشف
    boxes, confidences, class_ids, indexes = process_frame(frame)

    # رسم مربعات الإحاطة والتسميات على الإطار
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence = confidences[i]
            color = (0, 255, 0) if confidence > 0.5 else (0, 0, 255)  # لون المربع
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)  # رسم المربع
            cv2.putText(frame, f"{label} ({confidence*100:.2f}%)", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)  # كتابة التسمية

    # عرض الإطار المعالج
    cv2.imshow("Image", frame)
    key = cv2.waitKey(1)
    if key == 27:  # إذا تم الضغط على مفتاح ESC، نخرج من الحلقة
        break

# تحرير الكاميرا وإغلاق جميع النوافذ
cap.release()
cv2.destroyAllWindows()


استيراد المكتبات:

cv2 من OpenCV للعمل مع الصور والفيديو.
numpy للعمل مع المصفوفات.
threading لإنشاء وتشغيل الخيوط لتحسين الأداء.
تحميل YOLO:

تحميل شبكة YOLO والملفات اللازمة (الوزن والتكوين).
الحصول على أسماء الطبقات وتحديد الطبقات النهائية المستخدمة للكشف عن الكائنات.
تحميل أسماء الأصناف:

قراءة أسماء الأصناف من ملف نصي coco.names.
تهيئة الكاميرا:

فتح الكاميرا باستخدام OpenCV.
دالة process_frame:

معالجة الإطار للحصول على الكائنات المكتشفة.
تحويل الصورة إلى تنسيق blob المناسب للشبكة العصبية.
استخراج نتائج الكشف وتحليلها.
تطبيق NMS لتجنب التكرار في الكائنات المكتشفة.
حلقة المعالجة الرئيسية:

قراءة الإطارات من الكاميرا.
إنشاء وتشغيل خيط لمعالجة كل إطار.
استدعاء دالة المعالجة للحصول على نتائج الكشف.
رسم مربعات الإحاطة والتسميات على الإطار.
عرض الإطار المعالج.
الخروج من الحلقة عند الضغط على مفتاح ESC.
تحرير الموارد:

إغلاق الكاميرا وجميع النوافذ المفتوحة.
تحسينات مقترحة:
استخدام مكتبات GPU لتسريع عملية الكشف.
تحسين دقة الكشف باستخدام نماذج أحدث مثل YOLOv4 أو YOLOv5.
إضافة واجهة مستخدم رسومية لعرض الفيديو والتحكم فيه.
معالجة الإطارات بشكل متوازي لتحسين الأداء.